### Harmonic Quotations

Created by Vlad Praskurnin, this NB will identify harmonic repetitions and quotations, either
within a single piece or in a corpus.

It begins by making what we might call 'figured bass' representations of each offset in a composition, 
stacking up the intervals from lowest to highest:  B-T, B-A, B-C, etc.  (These use staff 
numbers, so in a four-voice piece 4-3, 4-2, 4-1, etc.

The stacked collections are saved a new column, then successive stacks are added to make long
'fingerprints' of harmonic passages.

Recurring passages are identified, and tracked to the pieces in which they appear.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [2]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_2.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_3.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_4.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0015_5.mei'])



Successfully imported https://crimproject.org/mei/CRIM_Model_0017.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0015_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0015_2.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0015_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0015_4.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0015_5.mei


In [10]:
quote_length = 10
func1 = ImportedPiece.harmonic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd', 'compound':False}, metadata=False)
func2 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func2, kwargs={'offset':True,'df': list_of_dfs})
cleaned_list = []
for harm in list_of_detail_index: #for each df sitting in the list applying these functions (for each df in list, can apply df functions, but not on list as a whole)
    harm = harm.fillna(method="pad")
    harm['figures'] = harm[harm.columns[:-2]].apply( # 0: takes all the columns
        lambda x: ''.join(x.astype(str)), #originally:  lambda x: ''.join(x.dropna().astype(str)),
        axis=1
    )
    harm = harm.loc[harm['figures'].shift() != harm['figures']]
    for i in range(1, quote_length):
        harm[f"figures{i}"] = harm['figures'].shift(-i) #or "figures"+str(i)
    harm["figures_combined"] = harm[harm.columns[-quote_length:]].apply( # 0: takes all the columns
        lambda x: '_'.join(x.astype(str)), #originally:  lambda x: ''.join(x.dropna().astype(str)),
        axis=1
    )
    harm = harm.drop(harm.columns[-quote_length:-1], axis=1)
    cleaned_list.append(harm)
harm_corpus = pd.concat(cleaned_list)
harm_corpus


6_5   6_4   6_3   6_2   6_1   5_4   5_3   5_2   5_1  \
Measure Beat Offset                                                         
1       1.0  0.0     Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest   
        4.5  7.0     Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest   
2       1.0  8.0     Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest   
        2.0  10.0    Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest   
        3.0  12.0    Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest  Rest   
...                   ...   ...   ...   ...   ...   ...   ...   ...   ...   
104     2.5  987.0      1     8     5     3     6     8     5     3     6   
        3.0  988.0      3     8     5     3     5     6     3     8     3   
        4.0  990.0      3     8     5     3     6     6     3     8     4   
        4.5  991.0      4     8     5     3     6     5     2     7     3   
105     1.0  992.0      8     3     8     5     3     3     8     5     3   

                      4_3   4_2   4_1   3_2   3_1 2_1  \
Measure Beat Offset                                     
1       1.0  0.0     Rest  Rest  Rest  Rest  Rest   8   
        4.5  7.0     Rest  Rest  Rest  Rest  Rest   7   
2       1.0  8.0     Rest  Rest  Rest  Rest  Rest   6   
        2.0  10.0    Rest  Rest  Rest  Rest  Rest   5   
        3.0  12.0    Rest  Rest  Rest  Rest  Rest   8   
...                   ...   ...   ...   ...   ...  ..   
104     2.5  987.0      5     3     6    -3     2   4   
        3.0  988.0      5     3     5    -3     1   3   
        4.0  990.0      5     3     6    -3     2   4   
        4.5  991.0      5     3     6    -3     2   4   
105     1.0  992.0      6     3     8    -4     3   6   

                                             Composer  \
Measure Beat Offset                                     
1       1.0  0.0                     Josquin Des Prés   
        4.5  7.0                     Josquin Des Prés   
2       1.0  8.0                     Josquin Des Prés   
        2.0  10.0                    Josquin Des Prés   
        3.0  12.0                    Josquin Des Prés   
...                                               ...   
104     2.5  987.0   Giovanni Pierluigi da Palestrina   
        3.0  988.0   Giovanni Pierluigi da Palestrina   
        4.0  990.0   Giovanni Pierluigi da Palestrina   
        4.5  991.0   Giovanni Pierluigi da Palestrina   
105     1.0  992.0   Giovanni Pierluigi da Palestrina   

                                             Title  \
Measure Beat Offset                                  
1       1.0  0.0                      Benedicta es   
        4.5  7.0                      Benedicta es   
2       1.0  8.0                      Benedicta es   
        2.0  10.0                     Benedicta es   
        3.0  12.0                     Benedicta es   
...                                            ...   
104     2.5  987.0   Missa Benedicta es: Agnus Dei   
        3.0  988.0   Missa Benedicta es: Agnus Dei   
        4.0  990.0   Missa Benedicta es: Agnus Dei   
        4.5  991.0   Missa Benedicta es: Agnus Dei   
105     1.0  992.0   Missa Benedicta es: Agnus Dei   

                                                               figures  \
Measure Beat Offset                                                      
1       1.0  0.0     RestRestRestRestRestRestRestRestRestRestRestRe...   
        4.5  7.0     RestRestRestRestRestRestRestRestRestRestRestRe...   
2       1.0  8.0     RestRestRestRestRestRestRestRestRestRestRestRe...   
        2.0  10.0    RestRestRestRestRestRestRestRestRestRestRestRe...   
        3.0  12.0    RestRestRestRestRestRestRestRestRestRestRestRe...   
...                                                                ...   
104     2.5  987.0                                    185368536536-324   
        3.0  988.0                                    385356383535-313   
        4.0  990.0                                    385366384536-324   
        4.5  991.0 

In [11]:
out = harm_corpus['figures_combined'].value_counts()[harm_corpus['figures_combined'].value_counts()>1].to_frame() #see if 10gram word comes back
#harm_corpus['figures_combined'].value_counts()[harm_corpus['figures_combined'].value_counts()>1].count()

In [12]:
#harm_corpus.to_csv('saved_csv/10gram_quotes_corpus.csv')

In [13]:
len(out)

22